In [1]:
import os
import nltk
import spacy
import es_core_news_sm
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fonchopinto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Inicializar el lematizador de spaCy
nlp = spacy.load('es_core_news_sm')

# Definir el directorio donde están los archivos .txt
directorio = 'DiscursosOriginales/'  # Reemplaza esto con tu directorio

# Lista para almacenar los contenidos de los archivos
contenidos = []

# Leer los textos y combinarlos en un solo corpus
for filename in os.listdir(directorio):
    if filename.endswith('.txt'):
        with open(os.path.join(directorio, filename), 'r', encoding='utf-8') as file:
            contenido = file.read()
            print(f'Archivo {filename} leído')

            # Tokenizar y lematizar el contenido
            tokens = word_tokenize(contenido.lower())# Convertir a minúsculas
            tokens_sin_stopwords = [word for word in tokens if word not in STOP_WORDS]
            tokens_sin_stopwords = ' '.join(tokens_sin_stopwords)

            # Convertir los lemas nuevamente a texto
            contenidos.append(tokens_sin_stopwords)


Archivo 91265.txt leído
Archivo 83107.txt leído
Archivo 79915.txt leído
Archivo 153144.txt leído
Archivo 94447.txt leído
Archivo 84898.txt leído
Archivo 149691.txt leído
Archivo 89628.txt leído
Archivo 85012.txt leído
Archivo 85006.txt leído
Archivo 80142.txt leído
Archivo 96720.txt leído
Archivo 75444.txt leído
Archivo 86296.txt leído
Archivo 91891.txt leído
Archivo 164307.txt leído
Archivo 100454.txt leído
Archivo 80552.txt leído
Archivo 136643.txt leído
Archivo 84907.txt leído
Archivo 102280.txt leído
Archivo 101951.txt leído
Archivo 71541.txt leído
Archivo 90595.txt leído
Archivo 76776.txt leído
Archivo 96872.txt leído
Archivo 96866.txt leído
Archivo 85952.txt leído
Archivo 164266.txt leído
Archivo 94863.txt leído
Archivo 73020.txt leído
Archivo 101415.txt leído
Archivo 80396.txt leído
Archivo 153186.txt leído
Archivo 75121.txt leído
Archivo 100509.txt leído
Archivo 136905.txt leído
Archivo 86042.txt leído
Archivo 163938.txt leído
Archivo 81705.txt leído
Archivo 102450.txt leído
Ar

In [74]:
print(contenidos[7])

tardes : general mario alberto rozas córdova , nombre gobierno , seguro interpretando compatriotas , deseo exitosa gestión mando carabineros chile . quería compartir reflexiones . , principal preocupación chilenos , probablemente seguir seguridad ciudadana , paz tranquilidad puedan desarrollar vidas libertad aspirar realización plena ciudadanos país . , consecuencia , prioridad compromiso gobiernos , honor encabezar , esté alcance paz tranquilidad compatriotas . frente enemigo formidable , poderoso , cruel implacable demostrado respeta , , cometer actos delincuencia , narcotráfico , crimen organizado terrorismo . , consecuencia , cumplir importante misión , obligación gobierno , labor carabineros chile absolutamente fundamental . quiero expresar , , profundo aprecio , reconocimiento gratitud realizan 60 mil carabineros ancho chile , visto ojos , desarrollan funciones entrega , abnegación , sacrificio realmente merece reconocimiento gratitud . visto elementos obligan profunda modernizac

In [75]:
model = Word2Vec(tokens, vector_size=300, window=2, min_count=1)

In [76]:
vectores_por_texto = []
for texto_tokens in contenidos:
    vectores_por_token = [model.wv[token] for token in texto_tokens if token in model.wv]
    if vectores_por_token:
        # Promediar los vectores para obtener un vector representativo del texto
        vector_promedio = sum(vectores_por_token) / len(vectores_por_token)
        vectores_por_texto.append(vector_promedio)
    else:
        # Si no hay tokens en el modelo, agregar un vector de ceros
        vectores_por_texto.append([0] * model.vector_size)

In [55]:
type(vectores_por_texto)

list

In [81]:
consulta = str(input("Ingrese su consulta: "))
tokens_consulta = word_tokenize(consulta.lower())
tokens_sin_stopwords = [word for word in tokens_consulta if word not in STOP_WORDS]
doc = ' '.join(tokens_sin_stopwords)
embedding_consulta = [model.wv[token] for token in doc if token in model.wv]
vector_promedio = sum(embedding_consulta) / len(embedding_consulta)
embedding_consulta = vector_promedio.reshape(1, -1)


ZeroDivisionError: division by zero

In [78]:
embedding_consulta

array([[ 0.00263781,  0.04363329, -0.00144523,  0.03996031, -0.02720808,
        -0.05909811,  0.0272177 ,  0.10108966, -0.01164784, -0.03081629,
         0.06764881, -0.05439822, -0.00310421,  0.09512483, -0.02483917,
        -0.04854558,  0.0750531 , -0.0087786 ,  0.01151189, -0.0130393 ,
         0.00227533, -0.00782   ,  0.00423429,  0.07260232, -0.01862298,
         0.00341676, -0.05142415,  0.05332867, -0.01657908, -0.04447757,
         0.03222081, -0.04373995,  0.03702968, -0.03100022, -0.03017275,
         0.01802315,  0.04306227, -0.05550192, -0.02385673, -0.01602083,
        -0.04856512,  0.00268789,  0.00520716, -0.02466968,  0.02303638,
         0.028199  , -0.04407751, -0.01759699,  0.01437938,  0.01421496,
        -0.01030769, -0.00319599, -0.07046477,  0.01722725,  0.01833652,
        -0.03254151,  0.01006808, -0.03681619,  0.04266152, -0.00582953,
        -0.0237365 ,  0.00152554,  0.01219406,  0.04015762, -0.02410178,
         0.03895801, -0.00224172,  0.06272057, -0.0

In [79]:
similitudes = []
counter = 0
for vector in vectores_por_texto:
    vector = vector.reshape(1, -1)
    similitud = cosine_similarity(embedding_consulta, vector)[0][0]
    similitudes.append((contenidos[counter], similitud))
    counter += 1

# Ordenar por similitud descendente
similitudes = sorted(similitudes, key=lambda x: x[1], reverse=True)

In [80]:
for texto, similitud in similitudes[:10]:  # Muestra los 5 textos más relevantes
    print(f'Similitud: {similitud}')
    print(texto)
    print('-' * 30)

Similitud: 0.999995768070221
: doloroso drama vivido miles miles niños país , vulnerables habiendo puestos manos organismos colaboradores queridos protegidos , abusados vulnerados . situación dejar indiferente , movernos acción . extraordinariamente gratificante emocionante amor , cariño , preocupación niños . minutos podido recorrer , sonrisas esperanzadoras , caritas alegres llenado fuerza voluntad seguir , vale millón argumentos millón palabras . habría gustado diputados senadores hubieran podido compartir mañana sonrisas , caras llenas esperanza , caritas pedían oportunidad niños niños , disfrutar niñez , inocencia , alegría vivir . misión impuesto fácil , niños fila , visitas , señora alicia , llenan corazón alegría alma voluntad seguir . acabamos firmar proyecto ley ; dirán “ proyecto ley ” . : proyecto ley permitir incrementar subvención reciben organismos colaboradores , sociedad protectora infancia , 124 años . , liderazgo ininterrumpido , allá ininterrumpido , realmente impre